In [1]:
# =========================================
# DESeq2 for pseudo-replicate pseudo-bulk
# =========================================

suppressPackageStartupMessages({
library(DESeq2)
library(tidyverse)
library(apeglm)
library(here)
})

In [2]:
setwd('..')

In [3]:
getwd()

[1] "C:/Users/yyyyx/Work/OC43_machine_learning/Regression_Modeling_of_OC43_Coronavirus_Viral_Burden_from_Host_Single-cell_Transcriptomes_Using_Explainable_Machine_Learning"

In [4]:
utils <- new.env()

sys.source(here::here("scripts", "utils.r"), envir = utils)

In [5]:
counts <- read_csv('./CSV/pb_counts.csv') %>%  as.data.frame()
coldata <- read_csv("./CSV/pb_meta.csv") %>%  as.data.frame()

New names:
• `` -> `...1`
Rows: 1614 Columns: 10
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): ...1
dbl (9): No infection_rep1, No infection_rep2, No infection_rep3, Low infect...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 9 Columns: 5
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): pseudo_sample, infection_group
dbl (3): replicate, n_cells, umi_sum

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
# Setting rownames
rownames(counts) <- counts[[1]]; counts[[1]] <- NULL
rownames(coldata) <- coldata[[1]]; coldata[[1]] <- NULL

In [7]:
dds <- DESeqDataSetFromMatrix(countData = counts,
                              colData = coldata,
                              design = ~ infection_group)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]



In [8]:
dds <- DESeq(dds)

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

final dispersion estimates

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This 

In [9]:
dds

class: DESeqDataSet 
dim: 1614 9 
metadata(1): version
assays(4): counts mu H cooks
rownames(1614): SAMD11 HES4 ... MT-ND5 MT-ND6
rowData names(26): baseMean baseVar ... deviance maxCooks
colnames(9): No infection_rep1 No infection_rep2 ... High
  infection_rep2 High infection_rep3
colData names(5): infection_group replicate n_cells umi_sum sizeFactor

In [10]:
resultsNames(dds)

[1] "Intercept"                                      
[2] "infection_group_Low.infection_vs_High.infection"
[3] "infection_group_No.infection_vs_High.infection"

In [11]:
utils$analyze_save(dds, "infection_group", "Low infection", "No infection", './Pseudobulk/Sig_Deg/', lfc_cutoff = 0.58)
utils$analyze_save(dds, "infection_group", "High infection", "No infection", './Pseudobulk/Sig_Deg/', lfc_cutoff = 0.58)
utils$analyze_save(dds, "infection_group", "High infection", "Low infection", './Pseudobulk/Sig_Deg/', lfc_cutoff = 0.58)

Processing: Low infection versus No infection ...

Warning message in write.csv(res_df, file = paste0(save_path, "DEG_results_", coefficient, :
"attempt to set 'col.names' ignored"
Warning message in write.csv(sig_deg, file = paste0(save_path, "Sig_DEG_", coefficient, :
"attempt to set 'col.names' ignored"
Done: Low infection | Found 765 DEGs

Processing: High infection versus No infection ...

Warning message in write.csv(res_df, file = paste0(save_path, "DEG_results_", coefficient, :
"attempt to set 'col.names' ignored"
Warning message in write.csv(sig_deg, file = paste0(save_path, "Sig_DEG_", coefficient, :
"attempt to set 'col.names' ignored"
Done: High infection | Found 975 DEGs

Processing: High infection versus Low infection ...

Warning message in write.csv(res_df, file = paste0(save_path, "DEG_results_", coefficient, :
"attempt to set 'col.names' ignored"
Warning message in write.csv(sig_deg, file = paste0(save_path, "Sig_DEG_", coefficient, :
"attempt to set 'col.names' ignor

In [12]:
suppressPackageStartupMessages({
  library(matrixStats) 
})

# ===== PCA (DESeq2 vst based) =====
vsd <- vst(dds, blind = TRUE)
mat <- assay(vsd)

pca <- prcomp(t(mat), center = TRUE, scale. = FALSE)

pca_df <- as.data.frame(pca$x)
pca_df$group  <- colData(dds)$condition
pca_df$sample <- colnames(mat)

var_ratio <- (pca$sdev ^ 2) / sum(pca$sdev ^ 2)
pc1_var   <- round(var_ratio[1] * 100, 1)
pc2_var   <- round(var_ratio[2] * 100, 1)

vr <- data.frame(PC = paste0("PC", seq_along(var_ratio)),
                 ratio = var_ratio)

In [13]:
write.csv(pca_df, './Pseudobulk/pca.csv', row.names = FALSE)
write.csv(vr, "./Pseudobulk/variance.csv", row.names = FALSE)

In [14]:
sessionInfo()

R version 4.2.3 (2023-03-15 ucrt)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 26200)

Matrix products: default

locale:
[1] LC_COLLATE=Korean_Korea.utf8  LC_CTYPE=Korean_Korea.utf8   
[3] LC_MONETARY=Korean_Korea.utf8 LC_NUMERIC=C                 
[5] LC_TIME=Korean_Korea.utf8    

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] here_1.0.2                  apeglm_1.20.0              
 [3] lubridate_1.9.3             forcats_1.0.1              
 [5] stringr_1.5.2               dplyr_1.1.4                
 [7] purrr_1.0.2                 readr_2.1.5                
 [9] tidyr_1.3.1                 tibble_3.2.1               
[11] ggplot2_3.5.1               tidyverse_2.0.0            
[13] DESeq2_1.38.3               SummarizedExperiment_1.28.0
[15] Biobase_2.58.0              MatrixGenerics_1.10.0      
[17] matrixStats_1.3.0           GenomicRanges_1.50.2   